In [1]:
import numpy as np
import numpy.matlib
import random
from src.lib.xeuclidean import XEuclidean

random.seed(1337)

In [2]:
def keygen(n, p):
    # Generating U
    u_seed = np.random.randint(p, size = (n, n))
    for i in range(0, n):
        u_seed[i, i] = random.randint(1, p - 1) if u_seed[i, i] == 0 else u_seed[i, i]
    u = np.matrix(np.triu(u_seed))
    
    # Generating L
    l_seed = np.random.randint(p, size = (n, n))
    for i in range(0, n):
        l_seed[i, i] = random.randint(1, p - 1) if l_seed[i, i] == 0 else l_seed[i, i]
    l = np.matrix(np.tril(l_seed))
    
#     Creating K = U * L
    encryption_key = l * u
    for i in range(0, n):
        for j in range(0, n):
            encryption_key[i, j] %= p
            
    # Creating K^{-1}
    bases = np.matlib.identity(n)
    
    y = []
    y.append(bases[:, 0] * XEuclidean().inverse_modulo(l[0, 0], p))
    for i in range(1, n):
        sum_ = 0
        for j in range(0, i):
            sum_ += (y[j] * l[i, j]) % p
        sum_ %= p
        y.append(np.array((((bases[:, i] - sum_)) * XEuclidean().inverse_modulo(l[i, i], p)) % p))
    
    y = np.matrix(np.column_stack(y)).transpose()

    x = [0] * n
    x[n - 1] = y[n - 1, :] * XEuclidean().inverse_modulo(u[n - 1, n - 1], p)
    for i in range(n - 2, -1, -1):
        sum_ = 0
        for j in range(i + 1, n):
            sum_ += (x[j] * u[i, j]) % p
        sum_ %= p
        
        x[i] = np.array((((y[i, :] - sum_)) * XEuclidean().inverse_modulo(u[i, i], p)) % p)
    
    decryption_key = np.matrix(np.reshape(x, (n, n)))

    return x, y, u, l, encryption_key, decryption_key

In [7]:
%%time
for i in range(0, 10):
    x, y, u, l, encryption_key, decryption_key = keygen(10, 2)

Wall time: 16.7 ms


In [8]:
encryption_key

matrix([[1, 0, 0, 1, 0, 1, 0, 1, 0, 1],
        [0, 1, 1, 1, 1, 0, 1, 0, 1, 0],
        [0, 1, 0, 0, 1, 0, 1, 1, 1, 0],
        [0, 0, 1, 0, 1, 0, 0, 0, 0, 1],
        [1, 0, 1, 1, 0, 1, 1, 0, 1, 0],
        [1, 1, 1, 1, 1, 0, 1, 1, 1, 1],
        [1, 0, 1, 1, 1, 0, 0, 0, 1, 1],
        [1, 1, 0, 1, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 1, 1, 0, 0, 0],
        [0, 1, 0, 1, 1, 0, 1, 0, 1, 0]])

In [9]:
decryption_key

matrix([[0., 0., 1., 0., 0., 1., 1., 1., 0., 1.],
        [1., 0., 1., 1., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 1., 0., 1., 0., 1., 1.],
        [0., 1., 0., 0., 1., 0., 0., 1., 1., 0.],
        [1., 0., 0., 1., 1., 1., 1., 0., 1., 1.],
        [1., 1., 0., 0., 0., 1., 0., 0., 1., 0.],
        [0., 1., 1., 1., 1., 0., 1., 0., 1., 0.],
        [0., 1., 1., 0., 1., 1., 1., 1., 1., 0.],
        [0., 0., 0., 1., 1., 0., 0., 1., 1., 1.]])

In [10]:
res = decryption_key * encryption_key

res %= 2

res

matrix([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])